In [159]:
from Solver import Solver
from InventorySimulator import InventorySimulator
import numpy as np
import pandas as pd
from Simulator import Simulator
from Data import Data
import os
import multiprocessing as mp
import time
import matplotlib.pyplot as plt

In [160]:
tests = [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 10), (2, 1), (2, 2), (2, 3),
         (2, 4), (2, 5), (2, 6), (2, 7), (2, 9), (2, 10), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6),
         (3, 7), (3, 8), (3, 9), (3, 10), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (4, 8),
         (4, 9), (4, 10), (7, 1), (7, 6), (7, 7), (8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (8, 6), (8, 7),
         (8, 8), (8, 9), (8, 10), (11, 3), (11, 9), (12, 1), (12, 2), (12, 3), (12, 4), (12, 5), (12, 6),
         (12, 7), (12, 8), (12, 9), (12, 10), (15, 2), (15, 4), (16, 1), (16, 2), (16, 3), (16, 4), (16, 5),
         (16, 6), (16, 7), (16, 8), (16, 9), (16, 10), (17, 1), (17, 2), (17, 3), (17, 4), (17, 5), (17, 6),
         (17, 7), (17, 8), (17, 9), (17, 10), (18, 1), (18, 2), (18, 3), (18, 4), (18, 5), (18, 6), (18, 7),
         (18, 8), (18, 9), (18, 10), (19, 1), (19, 2), (19, 3), (19, 4), (19, 5), (19, 6), (19, 7), (19, 8),
         (19, 9), (19, 10), (20, 1), (20, 2), (20, 3), (20, 4), (20, 5), (20, 6), (20, 7), (20, 8), (20, 9),
         (20, 10), (23, 1), (23, 2), (23, 3), (23, 4), (23, 6), (23, 10), (24, 1), (24, 3), (24, 4), (24, 5),
         (24, 6), (24, 7), (24, 8), (24, 9), (24, 10), (27, 2), (27, 3), (27, 4), (27, 10), (28, 1), (28, 2),
         (28, 4), (28, 5), (28, 6), (28, 7), (28, 8), (28, 9), (28, 10), (31, 1), (31, 2), (31, 3), (31, 4),
         (31, 6), (32, 1), (32, 2), (32, 3), (32, 5), (32, 6), (32, 7), (32, 8), (32, 9), (32, 10), (33, 1),
         (33, 2), (33, 3), (33, 4), (33, 5), (33, 6), (33, 7), (33, 8), (33, 9), (33, 10), (34, 1), (34, 2),
         (34, 3), (34, 4), (34, 5), (34, 6), (34, 7), (34, 8), (34, 9), (34, 10), (35, 1), (35, 2), (35, 3),
         (35, 4), (35, 5), (35, 6), (35, 7), (35, 8), (35, 9), (35, 10), (36, 1), (36, 2), (36, 3), (36, 4),
         (36, 5), (36, 6), (36, 7), (36, 8), (36, 9), (36, 10), (38, 4), (38, 6), (39, 1), (39, 2), (39, 3),
         (39, 4), (39, 5), (39, 6), (39, 7), (39, 8), (39, 9), (39, 10), (40, 1), (40, 2), (40, 3), (40, 4),
         (40, 5), (40, 6), (40, 7), (40, 8), (40, 9), (40, 10), (42, 10), (43, 2), (44, 1), (44, 2), (44, 3),
         (44, 4), (44, 5), (44, 6), (44, 7), (44, 8), (44, 9), (44, 10), (47, 2), (48, 1), (48, 2), (48, 3),
         (48, 4), (48, 5), (48, 6), (48, 7), (48, 8), (48, 9), (48, 10)]

In [ ]:
# find ISFCT X
def mp_worker(key):
    for it in range(20):
        X = Solver(key, 'isfct', iteration=it)
        file = 'isfct/' + str(key[0]) + '_' + str(key[1]) + '__' + str(it)+ '.x'
        np.savetxt(file, X.det_x, fmt='%d')
        print('Process {}  -  {}\t DONE'.format(key, it))

In [ ]:
p = multiprocessing.Pool(8)
p.map(mp_worker, tests)

In [ ]:
print('Done')

In [3]:
def mp_simul(k):
    data = Data(k)
    data.gen_scn(20, 2000)
    sfct = pd.read_csv('./results/SFCT_simulation.csv')
    sfct.columns = ['i', 'j', 'r', 0]
    bound = min(list(sfct[(sfct['i']==k[0])&(sfct['r']<20)&(sfct['j']==k[1])][0]))
    res = []
    for it in range(20): # 20
        try:
            file = './isfct/' + str(k[0]) + '_' + str(k[1]) + '__' + str(it) + '.x'
            X = InventorySimulator(file=file, data=data, bound=float(bound)*1.02)
            if X.status:
                res.append(X.obj)
            else:
                res.append('FALSE')
        except:
            res.append('EXCEPT')
    print(res)
    file_2 = 'isfct_simul/' + str(k[0]) + '_' + str(k[1]) + '.simul'
    np.savetxt(file_2, np.array(res), fmt='%s')

In [4]:
p_1 = mp.Pool(8)
p_1.map(mp_simul, tests)

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed valu

Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+1

Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
[9.914440482108063, 9.91898923035107, 9.581410947537321, 9.918989230350665, 9.577353081996904, 9.57735308199615, 9.914440482113193, 9.57735308199615, 9.918989230350665, 9.581410947537691, 9.91898923035107, 9.918989230350665, 9.577353081996904, 10.118278419225273, 9.914440482113193, 9.9

   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter

Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
[3.972836618511465, 4.196889458700946, 4.2847527506128, 4.0981762246857265, 4.509301197261516, 4.074661438625936, 4.260594772589917, 4.3265123772389344, 4.02350598040176, 3.9784772672626083, 4.257235257035385, 4.839127224628733, 4.103088820579952, 4.074661438625936, 4.103800474141046, 4.585550668748626, 4.224299340906432, 4.224930871338499, 5.3974936672666765, 3.972837214363409]
Change

...

Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
[5.28510199588057, 6.281878269015412, 5.922553571185883, 6.014275008033408, 6.933047803744099, 8.454443208967149, 6.312298267928487, 7.597894696880893, 8.042626769407455, 6.5041820137147965, 6.647060680679417, 5.285269557558571, 5.179265963618832, 5.7592220513306565, 6.80492975026322, 6.013545568215072, 5.513460931551382, 5.846394455737563, 6.50263527217286, 5.714032259599485]
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed 

   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter

Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+1

   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
******** problem (47, 2) is not solved optimally.
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
[11.15607686431855, 11.08181491470684, 16.352258408082022, 11.45251946042449, 13.513700715540253, 11.076223971170819, 10.98864445181303, 11.309507839550996, 12.542728280560887, 11.772136461566, 11.085851925217348, 11.723018009390529, 11.344546273779152, 13.774722770549054, 11.27011690071015, 10.919076209492758, 11.6430663365703, 11.024372096127166, 11.214044463407639, 11.735264474226632]
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [5]:
print('Done')

Done


In [6]:
def mp_simul_sfct(k):
    data = Data(k)
    data.gen_scn(20, 2000)
    sfct = pd.read_csv('./results/SFCT_simulation.csv')
    sfct.columns = ['i', 'j', 'r', 0]
    bound = min(list(sfct[(sfct['i']==k[0])&(sfct['r']<20)&(sfct['j']==k[1])][0]))
    res = []
    for it in range(20): # 20
        try:
            file = './sfct/' + str(k[0]) + '_' + str(k[1]) + '__' + str(it) + '.x'
            X = InventorySimulator(file=file, data=data, bound=float(bound)*1.02)
            if X.status:
                res.append(X.obj)
            else:
                res.append('FALSE')
        except:
            res.append('EXCEPT')
    print(res)
    file_2 = 'sfct_simul/' + str(k[0]) + '_' + str(k[1]) + '.simul'
    np.savetxt(file_2, np.array(res), fmt='%s')

In [7]:
p_2 = mp.Pool(8)
p_2.map(mp_simul_sfct, tests)

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed valu

   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
******** problem (7, 7) is not solved optimally.
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1

   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
[4.888333303969044, 4.0745253114436135, 3.9524129334466958, 3.955239717479806, 4.043398451768251, 4.201413427427043, 4.191904875066412, 3.9486667293507205, 4.124151719276249, 4.414143509374899, 4.339811831843905, 6.317965300516666, 4.110043157717715, 4.264014321159859, 4.23069920082973, 3.976434604383221, 4.047612503327148, 4.081463406698735, 4.186098779208447, 4.902000310239235]
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Pr

Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+1

Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
[7.077072481244796, 9.506173857258537, 'FALSE', 7.250743675110343, 7.498630327845982, 7.799670928969608, 8.25364856463667, 7.139275301670427, 8.255171971099564, 7.07461000078999, 8.915742728717722, 7.085927575585011, 7.065663320095739, 8.445156586790157, 8.253143757895963, 8.6991313177

...

Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
[10.068377600372088, 9.93448228881087, 10.665134499037599, 9.869786895016269, 10.628343639035492, 9.511916103324054, 9.508572058268982, 9.730000326045719, 9.729211890271344, 10.506649253256462, 9.79089027629977, 10.04472691400516, 12.413650769385228, 9.76420176703467, 9.486415483745645, 10.059844642736998, 9.694054765028064, 10.235853942406763, 10.069457645070358, 9.413814507279177]
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
[5

   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter

Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+1

   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [8]:
print('Done')

Done


In [161]:
def mp_simul_fct(k):
    data = Data(k)
    data.gen_scn(20, 2000)
    sfct = pd.read_csv('./results/SFCT_simulation.csv')
    sfct.columns = ['i', 'j', 'r', 0]
    bound = min(list(sfct[(sfct['i']==k[0])&(sfct['r']<20)&(sfct['j']==k[1])][0]))
    res = []
    try:
        file = './fct/' + str(k[0]) + '_' + str(k[1]) + '.x'
        X = InventorySimulator(file=file, data=data, bound=float(bound)*1.02)
        if X.status:
            res.append(X.obj)
        else:
            res.append('FALSE')
    except:
        res.append('EXCEPT')
    file_3 = 'fct_simul/' + str(k[0]) + '_' + str(k[1]) + '.simul'
    np.savetxt(file_3, np.array(res), fmt='%s')

In [162]:
p_2 = mp.Pool(8)
p_2.map(mp_simul_fct, tests)

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
******** problem (3, 7) is not solved optimally.
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Mi

KeyboardInterrupt: 

******** problem (3, 8) is not solved optimally.
******** problem (1, 2) is not solved optimally.
******** problem (2, 10) is not solved optimally.
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
******** problem (4, 8) is not solved optimally.
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
******** problem (3, 9) is not solved optimally.
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
******** problem (11, 3) is not solved optimally.
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Defaul

Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
******** problem (16, 2) is not solved optimally.
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
******** problem (23, 10) is not solved optimally.
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
******** problem (17, 1) is not solved optimally.
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
******** problem (20, 1) is not solved optimally.
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
******** problem (17, 8) is not solved optimally.
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimi

Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
******** problem (31, 4) is not solved optimally.
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
******** problem (27, 3) is not solved optimally.
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
******** problem (28, 7) is not solved optimally.
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
******** problem (31, 6) is not so

Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
******** problem (36, 10) is not solved optimally.
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
******** problem (42, 10) is not solved optimally.
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
******** problem (39, 5) is not solved optimally.
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
******** problem (40, 3) is not solved optimally.
Changed value of parameter TimeLimit to 120.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
******** problem (34, 1) is not solved optimally.
Changed value of parameter TimeLim